# Counting particles inside a volume with Numpy optimisations

In [1]:
import numpy as np
import time

## Overviewing components

In [2]:
# Set Test Parameters
n_values = 10
n_particles = 5
volume_limits = [[0.0, 0.4, 0.3],
                 [0.6, 1.0, 0.9]]

In [3]:
# Construct Array of Time Points
time_array = np.asarray(range(n_values))*10
time_array

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [4]:
# Construct Array of Spatial Coordinates for particles at each time point
space_values = np.random.rand(n_values, n_particles, 3)
space_values

array([[[0.56332216, 0.04929785, 0.85601641],
        [0.94219262, 0.64248388, 0.68361535],
        [0.22298487, 0.41120873, 0.54625255],
        [0.02640222, 0.8359959 , 0.60301224],
        [0.06217176, 0.64088751, 0.33746286]],

       [[0.13173725, 0.8869279 , 0.18751819],
        [0.5746025 , 0.96260179, 0.92663087],
        [0.78024811, 0.9147324 , 0.3273662 ],
        [0.93278141, 0.61557231, 0.43459998],
        [0.33786739, 0.3545003 , 0.76818211]],

       [[0.2845357 , 0.03570368, 0.93436926],
        [0.60163903, 0.60687898, 0.70585675],
        [0.84581434, 0.26307685, 0.86913245],
        [0.55454815, 0.81865447, 0.35217878],
        [0.12130464, 0.50462143, 0.12340973]],

       [[0.74626417, 0.90349853, 0.06912401],
        [0.69140675, 0.00267858, 0.34399927],
        [0.48764542, 0.32707927, 0.91068599],
        [0.82409355, 0.056215  , 0.93957332],
        [0.2006405 , 0.40242333, 0.27495607]],

       [[0.56089684, 0.3479281 , 0.94187986],
        [0.69384505, 0.932

In [5]:
# Check whether each particle is within the volume limits at each time point
particles_inside = ((space_values >= volume_limits[0]) & (space_values <= volume_limits[1])).all(axis=2)
particles_inside

array([[False, False,  True,  True,  True],
       [False, False, False, False, False],
       [False, False, False,  True, False],
       [False, False, False, False, False],
       [False, False,  True,  True, False],
       [ True,  True,  True, False, False],
       [False, False,  True, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False,  True, False,  True, False]])

In [6]:
# Count how many particles are inside the volume for each time point
num_particles_inside = particles_inside.sum(axis=1)
num_particles_inside

array([3, 0, 1, 0, 2, 3, 1, 0, 0, 2])

In [7]:
# Get indices for times with nonzero particles
nonzero_indices = num_particles_inside.nonzero()
nonzero_indices

(array([0, 2, 4, 5, 6, 9], dtype=int64),)

In [8]:
# Get times with nonzero particles
valid_times = time_array[nonzero_indices]
valid_times

array([ 0, 20, 40, 50, 60, 90])

In [9]:
# Extract numbers of particles for times with nonzero particles
valid_num_particles = num_particles_inside[num_particles_inside != 0]
valid_num_particles

array([3, 1, 2, 3, 1, 2])

In [10]:
# Put into final array (with additional indices array just for reference)
output = np.asarray([nonzero_indices, valid_times, valid_num_particles])
output

array([(array([0, 2, 4, 5, 6, 9], dtype=int64),),
       array([ 0, 20, 40, 50, 60, 90]), array([3, 1, 2, 3, 1, 2])],
      dtype=object)

## Speed Testing

In [11]:
volume_limits = [[0.0, 0.5, 0.3],
                 [0.4, 1.0, 0.8]]


def construct_input_array(n_times, n_particles):
    time_array = np.asarray(range(n_values))*10
    space_values = np.random.rand(n_values, n_particles, 3)
    return (time_array, space_values)


# Function as outlined above, using the numpy optimisations
def numpy_way(time_array, space_values):
    particles_inside = ((space_values >= volume_limits[0]) & (space_values <= volume_limits[1])).all(axis=2)
    num_particles_inside = particles_inside.sum(axis=1)
    nonzero_indices = num_particles_inside.nonzero()
    valid_times = time_array[nonzero_indices]
    valid_num_particles = num_particles_inside[num_particles_inside != 0]
    output = np.asarray([nonzero_indices, valid_times, valid_num_particles])
    return output


# Function as I would have originally done it, without Numpy optimisations
def python_way(time_array, space_values):
    valid_indices = []
    valid_particle_counts = []
    valid_times = []
    for index in range(len(space_values)):
        num_particles = 0
        for p in space_values[index]:
            in_volume = True
            for d in range(3):
                if ((p[d] < volume_limits[0][d]) or (p[d] > volume_limits[1][d])):
                    in_volume = False
                    break
            if in_volume:
                num_particles += 1
            
        if num_particles > 0:
            valid_indices.append(index)
            valid_particle_counts.append(num_particles)
            
    for index in valid_indices:
        valid_times.append(time_array[index])
        
    output = np.asarray([valid_indices, valid_times, valid_particle_counts])
    return output

In [12]:
# Test to make sure the outputs are the same
input_array = construct_input_array(10, 10)
print("Python Way:\n", python_way(*input_array))
print("Numpy Way:\n", numpy_way(*input_array))

Python Way:
 [[ 0  1  3  7  8  9]
 [ 0 10 30 70 80 90]
 [ 3  1  1  1  1  2]]
Numpy Way:
 [(array([0, 1, 3, 7, 8, 9], dtype=int64),) array([ 0, 10, 30, 70, 80, 90])
 array([3, 1, 1, 1, 1, 2])]


In [13]:
# Run time test over valid values
for t in range(3, 10):
    for p in range(3, 6):
        time_count = 10 ** t
        particle_count = 10 ** p
        print(f"time values: 10e{t}, num particles: 10e{p}")
        input_array = construct_input_array(time_count, particle_count)
        t1 = time.time()
        python_way(*input_array)
        t2 = time.time()
        numpy_way(*input_array)
        t3 = time.time()
        print(f"Python: {t2-t1:.2e} seconds")
        print(f"Numpy: {t3-t2:.2e} seconds\n")

time values: 10e3, num particles: 10e3
Python: 1.60e-02 seconds
Numpy: 9.96e-04 seconds

time values: 10e3, num particles: 10e4
Python: 1.26e-01 seconds
Numpy: 3.99e-03 seconds

time values: 10e3, num particles: 10e5
Python: 1.30e+00 seconds
Numpy: 3.29e-02 seconds

time values: 10e4, num particles: 10e3
Python: 1.40e-02 seconds
Numpy: 9.98e-04 seconds

time values: 10e4, num particles: 10e4
Python: 1.14e-01 seconds
Numpy: 2.99e-03 seconds

time values: 10e4, num particles: 10e5
Python: 1.34e+00 seconds
Numpy: 3.79e-02 seconds

time values: 10e5, num particles: 10e3
Python: 1.19e-02 seconds
Numpy: 0.00e+00 seconds

time values: 10e5, num particles: 10e4
Python: 1.51e-01 seconds
Numpy: 2.99e-03 seconds

time values: 10e5, num particles: 10e5
Python: 1.25e+00 seconds
Numpy: 3.09e-02 seconds

time values: 10e6, num particles: 10e3
Python: 9.97e-03 seconds
Numpy: 0.00e+00 seconds

time values: 10e6, num particles: 10e4
Python: 1.07e-01 seconds
Numpy: 2.99e-03 seconds

time values: 10e6, nu